## PrettyTensor

使用更加简便的包来构造卷积神经网络模型

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math

import prettytensor as pt

In [4]:
pt.__version__

'0.7.4'

In [5]:
# load data
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/MNIST/', one_hot=True)

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


In [6]:
# data dimensions
img_size = 28

img_size_flat = img_size * img_size

img_shape = (img_size, img_size)

num_channels = 1

num_classes = 10

In [7]:
# plot images sample
def plot_images(images, cls_true, cls_pred=None):
    assert len(images) == len(cls_true) == 9
    
    # Create figure with 3x3 sub-plots.
    fig, axes = plt.subplots(3, 3)
    fig.subplots_adjust(hspace=0.3, wspace=0.3)

    for i, ax in enumerate(axes.flat):
        # Plot image.
        ax.imshow(images[i].reshape(img_shape), cmap='binary')

        # Show true and predicted classes.
        if cls_pred is None:
            xlabel = "True: {0}".format(cls_true[i])
        else:
            xlabel = "True: {0}, Pred: {1}".format(cls_true[i], cls_pred[i])

        # Show the classes as the label on the x-axis.
        ax.set_xlabel(xlabel)
        
        # Remove ticks from the plot.
        ax.set_xticks([])
        ax.set_yticks([])
    
    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()

In [9]:
data.test.images[0].shape

(784,)

In [10]:
# input 
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')

x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])

y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')

y_true_cls = tf.argmax(y_true, dimension=1)

In [11]:
# Using PrettyTensor
x_pretty = pt.wrap(x_image)

In [14]:
with pt.defaults_scope(activation_fn=tf.nn.relu):
    # 核大小就是滤波器的大小
    y_pred, loss = x_pretty.\
        conv2d(kernel=5, depth=16, name='layer_conv1').\
        max_pool(kernel=2, stride=2).\
        conv2d(kernel=5, depth=36, name='layer_conv2').\
        flatten().\
        fully_connected(size=128, name='layer_fc1').\
        softmax_classifier(num_classes=num_classes, labels=y_true)

创建神经网络的每一层的过程交给PrettyTensor去做

那每一层每次迭代的权值变化就要我们从pt里面取出来，以便于可视化

In [16]:
def get_weights_variable(layer_name):
    
    with tf.variable_scope(layer_name, reuse=True):
        variable = tf.get_variable('weights')
    return variable

In [17]:
weights_conv1 = get_weights_variable(layer_name='layer_conv1')
weights_conv2 = get_weights_variable(layer_name='layer_conv2')

In [18]:
# Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss)

In [29]:
y_pred_cls = tf.argmax(y_pred, dimension=1)

data.test.cls = np.argmax(data.test.labels, axis=1)

In [20]:
correct_prediction = tf.equal(y_pred_cls, y_true_cls)

In [21]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [22]:
# compute
session = tf.Session()

session.run(tf.global_variables_initializer())

In [25]:
train_batch_size = 64

total_iterations = 0

def optimize(num_iterations):
    
    global total_iterations
    
    start_time = time.time()
    
    for i in range(total_iterations, total_iterations + num_iterations):
        
        x_batch, y_true_batch = data.train.next_batch(train_batch_size)
        
        feed_dict_train = {x: x_batch,
                           y_true: y_true_batch}
        
        session.run(optimizer, feed_dict=feed_dict_train)
        
        if i % 100 == 0:
            acc = session.run(accuracy, feed_dict=feed_dict_train)
            
            # Message for printing.
            msg = "Optimization Iteration: {0:>6}, Training Accuracy: {1:>6.1%}"
            
            print(msg.format(i + 1, acc))
    total_iterations += num_iterations
    
    end_time = time.time()
    
    time_diff = end_time - start_time
    
    print("Time useage: " + str(timedelta(seconds=int(round(time_diff)))))

In [27]:
# Split the test-set into smaller batches of this size.
test_batch_size = 256

def print_test_accuracy(show_example_errors=False,
                        show_confusion_matrix=False):

    # Number of images in the test-set.
    num_test = len(data.test.images)

    # Allocate an array for the predicted classes which
    # will be calculated in batches and filled into this array.
    cls_pred = np.zeros(shape=num_test, dtype=np.int)

    # Now calculate the predicted classes for the batches.
    # We will just iterate through all the batches.
    # There might be a more clever and Pythonic way of doing this.

    # The starting index for the next batch is denoted i.
    i = 0

    while i < num_test:
        # The ending index for the next batch is denoted j.
        j = min(i + test_batch_size, num_test)

        # Get the images from the test-set between index i and j.
        images = data.test.images[i:j, :]

        # Get the associated labels.
        labels = data.test.labels[i:j, :]

        # Create a feed-dict with these images and labels.
        feed_dict = {x: images,
                     y_true: labels}

        # Calculate the predicted class using TensorFlow.
        cls_pred[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)

        # Set the start-index for the next batch to the
        # end-index of the current batch.
        i = j

    # Convenience variable for the true class-numbers of the test-set.
    cls_true = data.test.cls

    # Create a boolean array whether each image is correctly classified.
    correct = (cls_true == cls_pred)

    # Calculate the number of correctly classified images.
    # When summing a boolean array, False means 0 and True means 1.
    correct_sum = correct.sum()

    # Classification accuracy is the number of correctly classified
    # images divided by the total number of images in the test-set.
    acc = float(correct_sum) / num_test

    # Print the accuracy.
    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, correct_sum, num_test))

#     # Plot some examples of mis-classifications, if desired.
#     if show_example_errors:
#         print("Example errors:")
#         plot_example_errors(cls_pred=cls_pred, correct=correct)

#     # Plot the confusion matrix, if desired.
#     if show_confusion_matrix:
#         print("Confusion Matrix:")
#         plot_confusion_matrix(cls_pred=cls_pred)

print_test_accuracy()

In [31]:
optimize(num_iterations=199)

Optimization Iteration:      1, Training Accuracy:  18.8%
Optimization Iteration:    101, Training Accuracy:  93.8%
Time useage: 0:00:23


In [32]:
print_test_accuracy()

Accuracy on Test-Set: 94.3% (9430 / 10000)


## 总结
PrettyTensor简化了神经网络构建过程

但与此同时，PrettyTensor还有一些缺点，设计问题还有文档的简陋

还有一些PrettyTensor的替代品，包括[TFLearn](https://github.com/tflearn/tflearn)和[Keras](https://github.com/fchollet/keras)